# Install MRjob

In [ ]:
! pip install mrjob

# Task 1: Get Word Count per Day
use composite key Map/Reduce

<date, word, count>

In [9]:
%%file task1.py

# file magic to create a file called task1.py
# file twitter-data.txt is of format: date, message, location, metadata
from mrjob.job import MRJob
import re
class MRWordFrequencyCount(MRJob):
    def mapper(self, _, line):
        # Split the line into fields 
        date, message,location,*other_metadata = line.split(',') # Extract the date and message

        date = date.strip()

        message = message.strip() # Extract words from the message
        words = re.findall(r'\w+', message.lower()) # Emit key-value pairs where the key is a composite key consisting of the date and word, 
        # and the value is 1

        for word in words:

            yield (date, word), 1

    def reducer(self, key, values): # Aggregate the counts for each word on each date
        yield key[0], (key[1], sum(values))
        
if __name__ == '__main__':
    MRWordFrequencyCount.run()

UsageError: Line magic function `%%file` not found.


In [ ]:
! python task1.py twitter-data.txt > task1_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\druzm\AppData\Local\Temp\task1.druzm.20240221.155920.431952
Running step 1 of 1...
job output is in C:\Users\druzm\AppData\Local\Temp\task1.druzm.20240221.155920.431952\output
Streaming final output from C:\Users\druzm\AppData\Local\Temp\task1.druzm.20240221.155920.431952\output...
Removing temp directory C:\Users\druzm\AppData\Local\Temp\task1.druzm.20240221.155920.431952...


# Task 2: Get total word count
re-use the output from Task 1 (word count per day) partially aggregated

In [104]:
%%file task2.py
from mrjob.job import MRJob

class MRTotalWordCount(MRJob):
    def mapper(self, _, line):
        _, word_count = line.split('\t')
        word, count = eval(word_count)
        yield word, int(count)

    def reducer(self, key, counts):
        yield key, sum(counts)

if __name__ == '__main__':
    MRTotalWordCount.run()


Overwriting task2.py


In [105]:
! python task2.py task1_output.txt > task2_output.txt

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\druzm\AppData\Local\Temp\task2.druzm.20240221.170108.149569
Running step 1 of 1...
job output is in C:\Users\druzm\AppData\Local\Temp\task2.druzm.20240221.170108.149569\output
Streaming final output from C:\Users\druzm\AppData\Local\Temp\task2.druzm.20240221.170108.149569\output...
Removing temp directory C:\Users\druzm\AppData\Local\Temp\task2.druzm.20240221.170108.149569...


In [107]:
# mapper test
with open('task1_output.txt', 'r') as file:
    task1_output = file.readlines()
    
def mapper_task1_output(line):
    _, word_count = line.split('\t')
    word, count = eval(word_count)
    return word, int(count)

for line in task1_output:
    print(mapper_task1_output(line))

('for', 1)
('pti', 2)
('vote', 1)
('wins', 1)
('pti', 1)
('supporter', 1)
('behindtheskipper', 1)
